# Ejemplos de group 

## Definición de dependencias

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import branca
import branca.colormap as cm
import os

## Lectura de los datos de entrada

In [2]:
#Se lee el contenido de la base de datos fusionada
inputpath ="/home/reyes/backup/retc20042022fusionada/retc_2004_2022_complete.csv"
joined = pd.read_csv(inputpath, sep=",",low_memory=False, encoding='utf-8')

## Actualizaciones en los valores previo al procesamiento

### Se reemplazan los valores vacios en la columna iarc_group

In [3]:
joined.fillna({"iarc_group":"NC"},inplace=True)

### Se convierten los valores de la columna cve_ent de int (1 a 32) a str ('01' a '32' )

In [4]:
joined["cve_ent_str"] = [str(x).zfill(2) for x in joined.cve_ent]


### Se convierten los valores de la columna cve_mun de int (0 a 999) a str ('00' a '999' )

In [5]:
joined["cve_mun_str"] = [str(x).zfill(3) for x in joined.cve_mun]

### Se cambia el valor de iarc_group * por Only RETC

In [6]:
#joined["iarc_group"] = [str(x).zfill(3) for x in joined.cve_mun]
joined.loc[joined["iarc_group"] == "*", "iarc_group"] = "NC"

## Primer ejemplo del group

### Se obtiene un subconjunto de columnas para el análisis

In [7]:
#Se clonan los datos de entrada
to_group = joined[[
    "anio", "cve_ent_str",'estado', 'cve_mun_str', 'municipio', 'sector', 'localidad', 'cas', 'sustancia', 'iarc_group', "finallat", "finallng",
    'em_agua', 'em_aire', 'em_suelo', 
    'tr_alcantarillado', 'tr_coprocesamiento', 'tr_dispfinal', 'tr_incineracion', 'tr_otros', 'tr_reciclado', 'tr_reutilizacion', 'tr_tratamiento',
]]


In [8]:
#Se realiza la primer a agrupacion
#joined.columns
#to_group.head()

### Agrupación por año

In [9]:
#grouped_by_year = to_group.groupby(["anio"])

#### Se agrega el valor de año a todas las filas

In [10]:
#to_group.anio.reset_index()

In [11]:
#grouped_by_year.first()
#grouped_by_year.get_group(2004)

In [16]:
#for namedf,group_df  in grouped_by_year:
#    df_temp = group_df.sort_values(by=["cve_ent_str", "cve_mun_str"])
    #print(df_temp.head())
    #df_temp.to_csv("gbyyear/{}_group.csv".format(df_temp['anio'].iloc[0]), index=False)
    #data.anio.reset_index()

grouped_by_iarc = to_group.groupby(["iarc_group"])

import folium


In [ ]:

COLORS = list(to_group["iarc_group"].unique())
colormap = cm.LinearColormap(colors=['lightblue','red'], index=[min(COLORS),max(COLORS)],vmin=min(COLORS),vmax=max(COLORS))
for namedf,group_df  in grouped_by_iarc:
    df_temp = group_df.sort_values(by=["cve_ent_str", "cve_mun_str"])
    print(df_temp['iarc_group'].iloc[0])
    #df_temp.to_csv("gby_year_ent/{}_{}_group.csv".format(df_temp['anio'].iloc[0], df_temp['cve_ent_str'].iloc[0]), index=False)
    #out_path = "%s%s/" %("to_publish/",df_temp['iarc_group'].iloc[0])
    out_path = "to_publish/{}".format(df_temp['iarc_group'].iloc[0])
    try:    
        os.mkdir(out_path)
    except FileExistsError:
        pass


    locat= [to_group["finallat"].median(), to_group["finallng"].median()]
    

    #iniciamos el recorrio los datos del grupo
    for index, location_info in group_df.iterrows():

        map_layer = folium.Map(location=locat, zoom_start=6, control_scale=True,prefer_canvas=True)
        class_label = location_info["iarc_group"]
        html = ""
        html += "<p>{}{}</p>".format(location_info["cas"],location_info["sustancia"])
        html += "<p>{}{}</p>".format("Aire",location_info["em_aire"])
        html += "<p>{}{}</p>".format("Agua",location_info["em_agua"])
        html += "<p>{}{}</p>".format("Suelo",location_info["em_suelo"])

        iframe = folium.IFrame(html,width=100,height=100)
        popup = folium.Popup(iframe,max_width=100)
        t =  folium.CircleMarker(location=[location_info["finallat"],location_info["finallng"]],
                        radius=4,
                        fill=True,
                        color=colormap(class_label),
                        popup=popup,
                        fill_color=colormap(class_label)).add_to(map_layer)
        

        try:
            image_path = '%s%s.html'%(out_path,class_label)
            #LOGER.error("GUARDANDO")
            map_layer.save(image_path)
            del map_layer
            #LOGER.error(image_path)
            #if MapType==2:
            #    break
        
        except Exception as e:
            print("Error")

    #locat= [df_temp["finallat"].median(), df_temp["finallng"].median()]


    
